In this notebook we demonstrate saving a `Tabular` as a pickle.

We first replicate all the work done up to the creation of the `to` object.

In [1]:
from fastai.tabular.all import *

comp = 'bluebook-for-bulldozers'
path = URLs.path(comp)

Path.BASE_PATH = path

df = pd.read_csv(path/'TrainAndValid.csv', engine='pyarrow', dtype=object).convert_dtypes(convert_string=False)
for colname, col_dtype in df.dtypes.items():
    if col_dtype != np.dtype('O'):
        continue
    df.loc[df[colname] == '', colname] = np.nan

sizes = 'Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact'
df['ProductSize'] = df['ProductSize'].astype('category')
df['ProductSize'].cat.set_categories(sizes, ordered=True)

dep_var = 'SalePrice'
df[dep_var] = np.log(df[dep_var])

df['saledate'] = pd.to_datetime(df['saledate'], format='%m/%d/%Y 0:00')

df = add_datepart(df, 'saledate')
' '.join(o for o in df.columns if o.startswith('sale'))

cont, cat = cont_cat_split(df, 1, dep_var=dep_var)
procs = [Categorify, FillMissing]

cond = (df.saleYear < 2011) | (df.saleMonth < 10)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx), list(valid_idx))

to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

/home/jhanschoo/.local/share/hatch/env/virtual/fai/Rcp_9cR7/fastbook/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/home/jhanschoo/.local/share/hatch/env/virtual/fai/Rcp_9cR7/fastbook/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method 

We save it as a pickle.

In [2]:
save_pickle(path/'to.pkl', to)

And then load it.

In [3]:
to = load_pickle(path/'to.pkl')